# Szemantikus Kernel az OpenBnB MCP szerver integrációjával

Ez a jegyzetfüzet bemutatja, hogyan lehet a Szemantikus Kernel-t használni az OpenBnB MCP szerverrel, hogy valódi Airbnb szállásokat keressünk az MCPStdioPlugin segítségével. Az LLM hozzáféréshez az Azure AI Foundry-t használja. A környezeti változók beállításához kövesse a [Beállítási leckét](/00-course-setup/README.md).


## Importálja a szükséges csomagokat


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Az MCP Plugin kapcsolat létrehozása

Kapcsolódni fogunk az [OpenBnB MCP szerverhez](https://github.com/openbnb-org/mcp-server-airbnb) az MCPStdioPlugin segítségével. Ez a szerver az Airbnb keresési funkciót biztosítja az @openbnb/mcp-server-airbnb csomagon keresztül.


## Az ügyfél létrehozása

Ebben a példában az Azure AI Foundry-t fogjuk használni LLM hozzáféréshez. Győződjön meg róla, hogy a környezeti változók megfelelően vannak beállítva.


## Környezet konfigurációja

Konfiguráld az Azure OpenAI beállításait. Győződj meg róla, hogy a következő környezeti változók be vannak állítva:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Az OpenBnB MCP integráció megértése

Ez a jegyzetfüzet a **valós OpenBnB MCP szerverhez** csatlakozik, amely tényleges Airbnb keresési funkciókat biztosít.

### Hogyan működik:

1. **MCPStdioPlugin**: Standard bemeneti/kimeneti kommunikációt használ az MCP szerverrel
2. **Valós NPM csomag**: Letölti és futtatja az `@openbnb/mcp-server-airbnb` csomagot az npx segítségével
3. **Élő adatok**: Valós Airbnb ingatlanadatokat ad vissza az API-król
4. **Funkciók felfedezése**: Az ügynök automatikusan felfedezi az MCP szerver által elérhető funkciókat

### Elérhető funkciók:

Az OpenBnB MCP szerver általában az alábbiakat biztosítja:
- **search_listings** - Airbnb ingatlanok keresése hely és kritériumok alapján
- **get_listing_details** - Részletes információk lekérése konkrét ingatlanokról
- **check_availability** - Elérhetőség ellenőrzése meghatározott dátumokra
- **get_reviews** - Vélemények lekérése ingatlanokról
- **get_host_info** - Információk lekérése az ingatlanok házigazdáiról

### Előfeltételek:

- **Node.js** telepítve a rendszereden
- **Internetkapcsolat** az MCP szerver csomag letöltéséhez
- **NPX** elérhetőség (a Node.js része)

### A kapcsolat tesztelése:

Az MCP szervert manuálisan tesztelheted az alábbi parancs futtatásával:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ez letölti és elindítja az OpenBnB MCP szervert, amelyhez a Semantic Kernel csatlakozik, hogy valós Airbnb adatokat érjen el.


## Az Agent futtatása az OpenBnB MCP szerverrel

Most futtatjuk az AI Agentet, amely csatlakozik az OpenBnB MCP szerverhez, hogy valódi Airbnb szállásokat keressen Stockholm városában 2 felnőtt és 1 gyerek számára. Nyugodtan módosítsd a `user_inputs` listát a keresési feltételek megváltoztatásához.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Összefoglaló  
Gratulálunk! Sikeresen létrehoztál egy AI ügynököt, amely a Model Context Protocol (MCP) segítségével valós szálláskeresést integrál:

Használt technológiák:  
- Semantic Kernel - Intelligens ügynökök építéséhez Azure OpenAI-val  
- Azure AI Foundry - LLM képességekhez és csevegés befejezéséhez  
- MCP (Model Context Protocol) - Szabványosított eszközintegrációhoz  
- OpenBnB MCP Server - Valós Airbnb keresési funkciókhoz  
- Node.js/NPX - Külső MCP szerver futtatásához  

Amit megtanultál:  
- MCP integráció: Semantic Kernel ügynökök csatlakoztatása külső MCP szerverekhez  
- Valós idejű adathozzáférés: Valós Airbnb ingatlanok keresése élő API-kon keresztül  
- Protokollkommunikáció: stdio kommunikáció használata az ügynök és az MCP szerver között  
- Funkciófelfedezés: MCP szerverek elérhető funkcióinak automatikus felfedezése  
- Streaming válaszok: Funkcióhívások valós idejű rögzítése és naplózása  
- HTML megjelenítés: Ügynökválaszok formázása stílusos táblázatokkal és interaktív kijelzőkkel  

Következő lépések:  
- További MCP szerverek integrálása (időjárás, repülőjáratok, éttermek)  
- Több ügynökből álló rendszer építése MCP és A2A protokollok kombinálásával  
- Egyedi MCP szerverek létrehozása saját adatforrásokhoz  
- Tartós beszélgetési memória megvalósítása munkamenetek között  
- Az ügynök telepítése Azure Functions-re MCP szerverek összehangolásával  
- Felhasználói hitelesítés és foglalási funkciók hozzáadása  

Az MCP architektúra fő előnyei:  
- Szabványosítás: Univerzális protokoll AI ügynökök és külső eszközök összekapcsolásához  
- Valós idejű adatok: Élő, naprakész információk elérése különböző szolgáltatásokból  
- Bővíthetőség: Új adatforrások és eszközök egyszerű integrációja  
- Interoperabilitás: Különböző AI keretrendszerekkel és ügynökplatformokkal működik  
- Felelősségek szétválasztása: Egyértelmű különbségtétel az AI logika és a külső adathozzáférés között  



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
